# ETL Project
### David Kloepper & Scott Otto
### Data Visualization Bootcamp, Cohort 3
### May 18th, 2019

In [1]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine
import os
import time

#yelp_api is a separate Python script from making Yelp API calls
import yelp_api

## Preparing data for API requests

In [2]:
#Read in inspection data CSV
inspection_data_df = pd.read_csv("food-inspections.csv")
inspection_data_df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2288615,SABROSO,SABROSO,2664160.0,Restaurant,Risk 1 (High),624 N ASHLAND AVE,CHICAGO,IL,60622.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.893250,-87.667356,"{'latitude': '41.893249885934495', 'longitude'...",24.0,21560.0,25.0,520.0,41.0
1,2288592,PINK TOWER MONTESSORI SCHOOL,PINK TOWER MONTESSORI SCHOOL,2457655.0,Children's Services Facility,Risk 1 (High),3420 N ELSTON AVE,CHICAGO,IL,60618.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.943078,-87.703898,"{'latitude': '41.943077977197895', 'longitude'...",2.0,21538.0,22.0,425.0,20.0
2,2288595,POLITAN ROW,POLITAN ROW,2621900.0,Restaurant,Risk 3 (Low),111 N ABERDEEN ST,CHICAGO,IL,60607.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.883370,-87.654261,"{'latitude': '41.883370342897926', 'longitude'...",41.0,14917.0,29.0,63.0,46.0
3,2288613,ANTHONY'S DINER,ANTHONY'S DINER,2664063.0,Restaurant,Risk 1 (High),7544 W ADDISON ST,CHICAGO,IL,60634.0,...,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.945226,-87.816074,"{'latitude': '41.945225909077436', 'longitude'...",39.0,22254.0,17.0,392.0,19.0
4,2288630,EL NUEVO PUERTO MARISCOS AND GRILL,EL NUEVO PUERTO MARISCOS AND GRILL,2664156.0,Restaurant,Risk 1 (High),2752 W LAWRENCE AVE,CHICAGO,IL,60625.0,...,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.968645,-87.698286,"{'latitude': '41.968644613011605', 'longitude'...",46.0,21849.0,6.0,28.0,24.0


In [3]:
#Filter out rows with license ID = 0
has_license_df = inspection_data_df[inspection_data_df['License #']>0]

#Filter out any rows that are not identified as Restaurants
restaurant_inspection_data_df = has_license_df[has_license_df['Facility Type']=="Restaurant"]
restaurant_inspection_data_df.reset_index(inplace=True, drop=True)
restaurant_inspection_data_df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2288615,SABROSO,SABROSO,2664160.0,Restaurant,Risk 1 (High),624 N ASHLAND AVE,CHICAGO,IL,60622.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.893250,-87.667356,"{'latitude': '41.893249885934495', 'longitude'...",24.0,21560.0,25.0,520.0,41.0
1,2288595,POLITAN ROW,POLITAN ROW,2621900.0,Restaurant,Risk 3 (Low),111 N ABERDEEN ST,CHICAGO,IL,60607.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.883370,-87.654261,"{'latitude': '41.883370342897926', 'longitude'...",41.0,14917.0,29.0,63.0,46.0
2,2288613,ANTHONY'S DINER,ANTHONY'S DINER,2664063.0,Restaurant,Risk 1 (High),7544 W ADDISON ST,CHICAGO,IL,60634.0,...,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.945226,-87.816074,"{'latitude': '41.945225909077436', 'longitude'...",39.0,22254.0,17.0,392.0,19.0
3,2288630,EL NUEVO PUERTO MARISCOS AND GRILL,EL NUEVO PUERTO MARISCOS AND GRILL,2664156.0,Restaurant,Risk 1 (High),2752 W LAWRENCE AVE,CHICAGO,IL,60625.0,...,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.968645,-87.698286,"{'latitude': '41.968644613011605', 'longitude'...",46.0,21849.0,6.0,28.0,24.0
4,2288623,D FRUTA LA VIDA LLC,D FRUTA LA VIDA,2398055.0,Restaurant,Risk 1 (High),3739 W FULLERTON AVE,CHICAGO,IL,60647.0,...,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.924443,-87.721033,"{'latitude': '41.924442829878934', 'longitude'...",15.0,22535.0,23.0,74.0,12.0


In [4]:
#Set the Inspection Date field to a datetime 
restaurant_inspection_data_df['Inspection Date'] = pd.to_datetime(restaurant_inspection_data_df['Inspection Date'])
#Return the most recent inspection row for each license ID by inspection date
idx = restaurant_inspection_data_df.groupby(['License #'])['Inspection Date'].transform(max) == restaurant_inspection_data_df['Inspection Date']

most_recent_inspection_data_df = restaurant_inspection_data_df[idx]
most_recent_inspection_data_df

C:\Users\dkloe\Anaconda3\envs\PythonWebMongo\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2288615,SABROSO,SABROSO,2664160.0,Restaurant,Risk 1 (High),624 N ASHLAND AVE,CHICAGO,IL,60622.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.893250,-87.667356,"{'latitude': '41.893249885934495', 'longitude'...",24.0,21560.0,25.0,520.0,41.0
1,2288595,POLITAN ROW,POLITAN ROW,2621900.0,Restaurant,Risk 3 (Low),111 N ABERDEEN ST,CHICAGO,IL,60607.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.883370,-87.654261,"{'latitude': '41.883370342897926', 'longitude'...",41.0,14917.0,29.0,63.0,46.0
2,2288613,ANTHONY'S DINER,ANTHONY'S DINER,2664063.0,Restaurant,Risk 1 (High),7544 W ADDISON ST,CHICAGO,IL,60634.0,...,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.945226,-87.816074,"{'latitude': '41.945225909077436', 'longitude'...",39.0,22254.0,17.0,392.0,19.0
3,2288630,EL NUEVO PUERTO MARISCOS AND GRILL,EL NUEVO PUERTO MARISCOS AND GRILL,2664156.0,Restaurant,Risk 1 (High),2752 W LAWRENCE AVE,CHICAGO,IL,60625.0,...,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.968645,-87.698286,"{'latitude': '41.968644613011605', 'longitude'...",46.0,21849.0,6.0,28.0,24.0
4,2288623,D FRUTA LA VIDA LLC,D FRUTA LA VIDA,2398055.0,Restaurant,Risk 1 (High),3739 W FULLERTON AVE,CHICAGO,IL,60647.0,...,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.924443,-87.721033,"{'latitude': '41.924442829878934', 'longitude'...",15.0,22535.0,23.0,74.0,12.0
5,2288589,LAS ISLAS MARIAS,LAS ISLAS MARIAS,1095992.0,Restaurant,Risk 1 (High),4770 W GRAND AVE,CHICAGO,IL,60639.0,...,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.913999,-87.745864,"{'latitude': '41.91399920930635', 'longitude':...",7.0,22615.0,26.0,62.0,44.0
6,2288628,TOM AND JERRY'S,TOM AND JERRY'S,2664659.0,Restaurant,Risk 1 (High),6359 W MONTROSE AVE,CHICAGO,IL,60634.0,...,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.960108,-87.786742,"{'latitude': '41.960108059177585', 'longitude'...",25.0,22254.0,15.0,131.0,19.0
7,2288643,RANGOLI,RANGOLI,2506262.0,Restaurant,Risk 1 (High),2421 W NORTH AVE,CHICAGO,IL,60647.0,...,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.910200,-87.688125,"{'latitude': '41.91019989714011', 'longitude':...",24.0,22535.0,25.0,302.0,41.0
8,2288612,DEVIL DAWGS,DEVIL DAWGS,2622063.0,Restaurant,Risk 1 (High),939 W BELMONT AVE,CHICAGO,IL,60657.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.939813,-87.653091,"{'latitude': '41.939813147048625', 'longitude'...",38.0,4449.0,57.0,681.0,25.0
9,2288531,BIRRIERIA TAPATIA,BIRRIERIA TAPATIA,2617403.0,Restaurant,Risk 1 (High),2861 W CERMAK RD,CHICAGO,IL,60623.0,...,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.851758,-87.698023,"{'latitude': '41.8517576674455', 'longitude': ...",29.0,21569.0,32.0,203.0,1.0


In [5]:
#Include only inspections that occurred since the start or 2018
inspections_since2018_df = most_recent_inspection_data_df[most_recent_inspection_data_df['Inspection Date']>="2018-01-01"]
inspections_since2018_df

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2288615,SABROSO,SABROSO,2664160.0,Restaurant,Risk 1 (High),624 N ASHLAND AVE,CHICAGO,IL,60622.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.893250,-87.667356,"{'latitude': '41.893249885934495', 'longitude'...",24.0,21560.0,25.0,520.0,41.0
1,2288595,POLITAN ROW,POLITAN ROW,2621900.0,Restaurant,Risk 3 (Low),111 N ABERDEEN ST,CHICAGO,IL,60607.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.883370,-87.654261,"{'latitude': '41.883370342897926', 'longitude'...",41.0,14917.0,29.0,63.0,46.0
2,2288613,ANTHONY'S DINER,ANTHONY'S DINER,2664063.0,Restaurant,Risk 1 (High),7544 W ADDISON ST,CHICAGO,IL,60634.0,...,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.945226,-87.816074,"{'latitude': '41.945225909077436', 'longitude'...",39.0,22254.0,17.0,392.0,19.0
3,2288630,EL NUEVO PUERTO MARISCOS AND GRILL,EL NUEVO PUERTO MARISCOS AND GRILL,2664156.0,Restaurant,Risk 1 (High),2752 W LAWRENCE AVE,CHICAGO,IL,60625.0,...,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.968645,-87.698286,"{'latitude': '41.968644613011605', 'longitude'...",46.0,21849.0,6.0,28.0,24.0
4,2288623,D FRUTA LA VIDA LLC,D FRUTA LA VIDA,2398055.0,Restaurant,Risk 1 (High),3739 W FULLERTON AVE,CHICAGO,IL,60647.0,...,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.924443,-87.721033,"{'latitude': '41.924442829878934', 'longitude'...",15.0,22535.0,23.0,74.0,12.0
5,2288589,LAS ISLAS MARIAS,LAS ISLAS MARIAS,1095992.0,Restaurant,Risk 1 (High),4770 W GRAND AVE,CHICAGO,IL,60639.0,...,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.913999,-87.745864,"{'latitude': '41.91399920930635', 'longitude':...",7.0,22615.0,26.0,62.0,44.0
6,2288628,TOM AND JERRY'S,TOM AND JERRY'S,2664659.0,Restaurant,Risk 1 (High),6359 W MONTROSE AVE,CHICAGO,IL,60634.0,...,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.960108,-87.786742,"{'latitude': '41.960108059177585', 'longitude'...",25.0,22254.0,15.0,131.0,19.0
7,2288643,RANGOLI,RANGOLI,2506262.0,Restaurant,Risk 1 (High),2421 W NORTH AVE,CHICAGO,IL,60647.0,...,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.910200,-87.688125,"{'latitude': '41.91019989714011', 'longitude':...",24.0,22535.0,25.0,302.0,41.0
8,2288612,DEVIL DAWGS,DEVIL DAWGS,2622063.0,Restaurant,Risk 1 (High),939 W BELMONT AVE,CHICAGO,IL,60657.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.939813,-87.653091,"{'latitude': '41.939813147048625', 'longitude'...",38.0,4449.0,57.0,681.0,25.0
9,2288531,BIRRIERIA TAPATIA,BIRRIERIA TAPATIA,2617403.0,Restaurant,Risk 1 (High),2861 W CERMAK RD,CHICAGO,IL,60623.0,...,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.851758,-87.698023,"{'latitude': '41.8517576674455', 'longitude': ...",29.0,21569.0,32.0,203.0,1.0


In [6]:
inspections_since2018_df.columns

Index(['Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Facility Type',
       'Risk', 'Address', 'City', 'State', 'Zip', 'Inspection Date',
       'Inspection Type', 'Results', 'Violations', 'Latitude', 'Longitude',
       'Location', 'Historical Wards 2003-2015', 'Zip Codes',
       'Community Areas', 'Census Tracts', 'Wards'],
      dtype='object')

In [7]:
#Rename columns to align to database fields
cleaned_inspections_df = inspections_since2018_df.rename(columns={"Inspection ID": "inspection_id",
"DBA Name": "dba_name",
"AKA Name": "aka_name",
"License #": "license_no",
"Facility Type": "facility_type",
"Risk": "risk",
"Address": "address",
"City": "city",
"State": "state",
"Zip": "zip",
"Inspection Date": "inspection_date",
"Inspection Type": "inspection_type",
"Results": "results",
"Violations": "violations",
"Latitude": "latitude",
"Longitude": "longitude",
"Location": "location",
"Historical Wards 2003-2015": "historical_wards",
"Zip Codes": "zip_codes",
"Community Areas": "community_areas",
"Census Tracts": "census_tracts",
"Wards": "wards"
})

cleaned_inspections_df.head()

,inspection_id,dba_name,aka_name,license_no,facility_type,risk,address,city,state,zip,...,results,violations,latitude,longitude,location,historical_wards,zip_codes,community_areas,census_tracts,wards
0,2288615,SABROSO,SABROSO,2664160.0,Restaurant,Risk 1 (High),624 N ASHLAND AVE,CHICAGO,IL,60622.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.893250,-87.667356,"{'latitude': '41.893249885934495', 'longitude'...",24.0,21560.0,25.0,520.0,41.0
1,2288595,POLITAN ROW,POLITAN ROW,2621900.0,Restaurant,Risk 3 (Low),111 N ABERDEEN ST,CHICAGO,IL,60607.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.883370,-87.654261,"{'latitude': '41.883370342897926', 'longitude'...",41.0,14917.0,29.0,63.0,46.0
2,2288613,ANTHONY'S DINER,ANTHONY'S DINER,2664063.0,Restaurant,Risk 1 (High),7544 W ADDISON ST,CHICAGO,IL,60634.0,...,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.945226,-87.816074,"{'latitude': '41.945225909077436', 'longitude'...",39.0,22254.0,17.0,392.0,19.0
3,2288630,EL NUEVO PUERTO MARISCOS AND GRILL,EL NUEVO PUERTO MARISCOS AND GRILL,2664156.0,Restaurant,Risk 1 (High),2752 W LAWRENCE AVE,CHICAGO,IL,60625.0,...,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.968645,-87.698286,"{'latitude': '41.968644613011605', 'longitude'...",46.0,21849.0,6.0,28.0,24.0
4,2288623,D FRUTA LA VIDA LLC,D FRUTA LA VIDA,2398055.0,Restaurant,Risk 1 (High),3739 W FULLERTON AVE,CHICAGO,IL,60647.0,...,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.924443,-87.721033,"{'latitude': '41.924442829878934', 'longitude'...",15.0,22535.0,23.0,74.0,12.0


In [8]:
#Export the dataframe to CSV so it can be read in later
if not os.path.isfile('restaurant_inspection_data.csv'):
   cleaned_inspections_df.to_csv('restaurant_inspection_data.csv', header='column_names', index=False)
else: # else it exists so append without writing the header
   cleaned_inspections_df.to_csv('restaurant_inspection_data.csv', mode='a', header=False, index=False)

In [9]:
#api_prep_df = restaurant_inspection_data_df.groupby(['License #',"DBA Name",'Address','City','State']).count()
#api_prep_df.reset_index(inplace=True)
#Filter the columns that are relevant to making the API calls
for_api_df = inspections_since2018_df.filter(['License #','DBA Name','Address','City','State'], axis=1)
for_api_df.head()

,License #,DBA Name,Address,City,State
0,2664160.0,SABROSO,624 N ASHLAND AVE,CHICAGO,IL
1,2621900.0,POLITAN ROW,111 N ABERDEEN ST,CHICAGO,IL
2,2664063.0,ANTHONY'S DINER,7544 W ADDISON ST,CHICAGO,IL
3,2664156.0,EL NUEVO PUERTO MARISCOS AND GRILL,2752 W LAWRENCE AVE,CHICAGO,IL
4,2398055.0,D FRUTA LA VIDA LLC,3739 W FULLERTON AVE,CHICAGO,IL


In [10]:
#Export the dataframe to CSV so it can be read in later
if not os.path.isfile('api_call_records.csv'):
   for_api_df.to_csv('api_call_records.csv', header='column_names', index=False)
else: # else it exists so append without writing the header
   for_api_df.to_csv('api_call_records.csv', mode='a', header=False, index=False)

## Running the API Requests

In [11]:
#Read in CSV for API calls. Since Yelp API only accepts 5000 calls per day, and two calls are made for each business,
# read in only 2500 rows at a time. Adjust commented rows accordingly to read all data
list_for_api_df = pd.read_csv("api_call_records.csv", header=0, nrows=2500)
#list_for_api_df = pd.read_csv("restaurants_only.csv", header=0 ,skiprows=range(1, 2501), nrows=2500)
#list_for_api_df = pd.read_csv("restaurants_only.csv", header=0 ,skiprows=range(1, 5001), nrows=2500)
#list_for_api_df = pd.read_csv("restaurants_only.csv", header=0 ,skiprows=range(1, 7501), nrows=2500)
list_for_api_df.head()

,License #,DBA Name,Address,City,State
0,2664160.0,SABROSO,624 N ASHLAND AVE,CHICAGO,IL
1,2621900.0,POLITAN ROW,111 N ABERDEEN ST,CHICAGO,IL
2,2664063.0,ANTHONY'S DINER,7544 W ADDISON ST,CHICAGO,IL
3,2664156.0,EL NUEVO PUERTO MARISCOS AND GRILL,2752 W LAWRENCE AVE,CHICAGO,IL
4,2398055.0,D FRUTA LA VIDA LLC,3739 W FULLERTON AVE,CHICAGO,IL


In [12]:
#Make API requests to Yelp API, referencing functions in external Python file
#Create dictionary of License ID to Yelp ID
#Create dictionary of returned Yelp data for each business

responses = []
licenseIDs = []

for row in list_for_api_df.iterrows():

    name = row[1]["DBA Name"]
    address = row[1]["Address"]
    city = row[1]["City"]
    state = row[1]["State"]
    country = 'US'
    licenseNum = row[1]["License #"]
    
    response = (yelp_api.match_query_api(name, address, city, state, country))
    
    inspectionName = row[1]["DBA Name"]
    inspectionAddress = row[1]["Address"]
    
    if response is not None:
        responses.append(response)
        
        yelpID = response['id']

        yelpName = response['name']
        yelpAddress = response['location']['address1']
    
        licenseID = {
                        "License #":licenseNum,
                        "Yelp ID":yelpID,
                        "Inspection Name":inspectionName,
                        "Inspection Address":inspectionAddress,
                        "Yelp Name":response['name'],
                        "Yelp Address":response['location']['address1']
                    }

    else:
        licenseID = {
                "License #":licenseNum,
                    "Yelp ID":0,
                    "Inspection Name":inspectionName,
                    "Inspection Address":inspectionAddress,
                    "Yelp Name":"",
                    "Yelp Address":""
                }
        
    
    licenseIDs.append(licenseID)
    
    time.sleep(.100)

Querying https://api.yelp.com/v3/businesses/matches ...
No businesses for SABROSO in 624 N ASHLAND AVE  found.
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/rsZ6Qd_lxW2NEKK5vy8ZVg ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/9Gei7OFl8mtKBzsSQvgQqQ ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/VS6BQRqCm5jZ-7jXMTwfSg ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/hv9IJpOjOkigqN7n8BeS4A ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/k6-CqrC7IiIsMpypP4vaZw ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/hmX4VH9WgDrEL9HJKcCL5A ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/1GiybV2faVpTt1zw13CzLA ...


Querying https://api.yelp.com/v3/businesses/IXwvEhZWkVR_bDfZs7dbhg ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/fGNGjLrTKb75W94DXiWCxw ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/J-jDDJYyPPsLg84jbJntfA ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/3K8whWvhWypsf44WU1yBjw ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/hmX4VH9WgDrEL9HJKcCL5A ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/LDkDsttuSBvlYQftP2xmSQ ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/Gbu8REboNkY_R4xXDJeLYg ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/KwbhZi_V-d2NfiX6gRu9pw ...
Querying https://api.yelp.com/v3/busines

Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/LrTJPD0wO8L_zBbWwzbirg ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/lPq5dWY57QyekebhvnVlrA ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/xHUnA-mMNd1aiF33g5XMNQ ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/S8RSmrV3oCptnTdimMYVBg ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/MFe_fnDkGdsUk9nV5U2VNg ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/MCxb7FbvsCQGYuapmmjlHA ...
Querying https://api.yelp.com/v3/businesses/matches ...
No businesses for LA ESQUINA DEL SABOR INC. in 4224 W 31ST ST  found.
Querying https://api.yelp.com/v3/businesses/matches ...
No businesses for UNIVERSAL ENTERTAINMENT CENTER in 1190

Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/aR9rgMDZSJl2JyxrW8q-nQ ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/4XQCW2YUdcfwYxl-v25-aw ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/mT_uFVYapK8xG7RZWh6aBg ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/GUwtdixuzA8u7EJHfgqRgA ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/PNVshX58m5A6hNAjUIe3AQ ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/PibX-YlN-xzJAk213QA0eg ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/zv6Np3a15fxtn8isqCFGPg ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/s8Dxq7Exy8D

Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/HU0YL7Hy98rYW1nrBFk5Qw ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/43eT02PoFWavFbxe4FzZzg ...
Querying https://api.yelp.com/v3/businesses/matches ...
No businesses for OAATO FOODS INC. in 7118 N CLARK ST  found.
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/dVFaT09phIIqPH_ss69I7Q ...
Querying https://api.yelp.com/v3/businesses/matches ...
No businesses for BEST TASTY SUB in 3723 W CHICAGO AVE  found.
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/zqhFTUFLmZgUBknFYGDxLg ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/J9WfLOHSTcXaT_9kz2gq0Q ...
Querying https://api.yelp.com/v3/businesses/matches ...
No businesses for LAILA'S GRAND GAME ROOM INC in 7110 W GRAND AVE  found

Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/WwwLeItKFssp7a9_9OnAPQ ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/8ogBepUZsGZ5CYuuP5h-dg ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/vfF4snBFXSxzebcsDjv6Fg ...
Querying https://api.yelp.com/v3/businesses/matches ...
No businesses for TAQUERIA EL JEFE LOCO CO. in 4646 W CHICAGO AVE  found.
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/5I9mVLHZupksTqU0BsP9Mw ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/A_og6lFgn8pG5jecJuzeUA ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/5tqU8J504L7AMTrXVHCtzw ...
Querying https://api.yelp.com/v3/businesses/matches ...
No businesses for David Jenson - Personal Chef ,Ltd 

Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/GP7p-YabjlbVxxdZS4PDCA ...
Querying https://api.yelp.com/v3/businesses/matches ...
No businesses for FARM BAR EDGEWATER in 1133 W GRANVILLE AVE  found.
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/Ivw59B7y_hobqdvrzpGz6A ...
Querying https://api.yelp.com/v3/businesses/matches ...
No businesses for ILLINOIS CENTER FOR REHABILITATION AND EDUCATION W in 1151 S WOOD ST  found.
Querying https://api.yelp.com/v3/businesses/matches ...
No businesses for NANDO'S PERI-PERI in 150 W ROOSEVELT RD  found.
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/ZTdXY7KDRbNdq0W20Q8kYQ ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/businesses/X4b9UHzAbeiBTqmvqQO9sw ...
Querying https://api.yelp.com/v3/businesses/matches ...
Querying https://api.yelp.com/v3/busin

ConnectionError: HTTPSConnectionPool(host='api.yelp.com', port=443): Max retries exceeded with url: /v3/businesses/matches?name=de+-+li+-+cious+cafe&address1=3827+N+LINCOLN+AVE+&city=CHICAGO&state=IL&country=US&limit=1&match_threshold=default (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000205897FF320>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [13]:
#Convert license id/Yelp Id dictionary to dataframe
licenseIDs_df = pd.DataFrame(licenseIDs)
licenseIDs_df.head()

,Inspection Address,Inspection Name,License #,Yelp Address,Yelp ID,Yelp Name
0,624 N ASHLAND AVE,SABROSO,2664160.0,,0,
1,111 N ABERDEEN ST,POLITAN ROW,2621900.0,111 N Aberdeen St,rsZ6Qd_lxW2NEKK5vy8ZVg,Politan Row Chicago
2,7544 W ADDISON ST,ANTHONY'S DINER,2664063.0,7544 W Addison St,9Gei7OFl8mtKBzsSQvgQqQ,Rick's Diner
3,2752 W LAWRENCE AVE,EL NUEVO PUERTO MARISCOS AND GRILL,2664156.0,2752 W Lawrence Ave,VS6BQRqCm5jZ-7jXMTwfSg,El Puerto Mariscos & Grill
4,3739 W FULLERTON AVE,D FRUTA LA VIDA LLC,2398055.0,3739 W Fullerton Ave,hv9IJpOjOkigqN7n8BeS4A,D Fruta La Vida


In [14]:
#Reorder the rows in the dataframe
cleaned_licenseIDs_df = licenseIDs_df.filter(['License #', 'Yelp ID', 'Inspection Name', 'Inspection Address', 'Yelp Name', 'Yelp Address'], axis=1)

#Rename the fields to align to database field names
cleaned_licenseIDs_df = cleaned_licenseIDs_df.rename(columns={"License #": "license_no",
                                                             "Yelp ID": "yelp_id", 
                                                             "Inspection Name":"inspection_name",
                                                            'Inspection Address':"inspection_address",
                                                            'Yelp Name':"yelp_name",
                                                            'Yelp Address':"yelp_address"})
cleaned_licenseIDs_df.head()

,license_no,yelp_id,inspection_name,inspection_address,yelp_name,yelp_address
0,2664160.0,0,SABROSO,624 N ASHLAND AVE,,
1,2621900.0,rsZ6Qd_lxW2NEKK5vy8ZVg,POLITAN ROW,111 N ABERDEEN ST,Politan Row Chicago,111 N Aberdeen St
2,2664063.0,9Gei7OFl8mtKBzsSQvgQqQ,ANTHONY'S DINER,7544 W ADDISON ST,Rick's Diner,7544 W Addison St
3,2664156.0,VS6BQRqCm5jZ-7jXMTwfSg,EL NUEVO PUERTO MARISCOS AND GRILL,2752 W LAWRENCE AVE,El Puerto Mariscos & Grill,2752 W Lawrence Ave
4,2398055.0,hv9IJpOjOkigqN7n8BeS4A,D FRUTA LA VIDA LLC,3739 W FULLERTON AVE,D Fruta La Vida,3739 W Fullerton Ave


In [15]:
#Export to CSV for import at a later time
if not os.path.isfile('id_data.csv'):
   cleaned_licenseIDs_df.to_csv('id_data.csv', header='column_names', index=False)
else: # else it exists so append without writing the header
   cleaned_licenseIDs_df.to_csv('id_data.csv', mode='a', header=False, index=False)

In [16]:
#Convert Yelp business dictionary to a dataframe
responses_df = pd.DataFrame.from_records(responses)
responses_df.head()

,alias,categories,coordinates,display_phone,hours,id,image_url,is_claimed,is_closed,location,messaging,name,phone,photos,price,rating,review_count,special_hours,transactions,url
0,politan-row-chicago-chicago-2,"[{'alias': 'food_court', 'title': 'Food Court'}]","{'latitude': 41.8832451, 'longitude': -87.654196}",(312) 278-3040,"[{'open': [{'is_overnight': False, 'start': '0...",rsZ6Qd_lxW2NEKK5vy8ZVg,https://s3-media2.fl.yelpcdn.com/bphoto/LYBaBR...,True,False,"{'address1': '111 N Aberdeen St', 'address2': ...",{'url': 'https://www.yelp.com/raq/politan-row-...,Politan Row Chicago,+13122783040,[https://s3-media2.fl.yelpcdn.com/bphoto/LYBaB...,NaN,4.5,9,NaN,[],https://www.yelp.com/biz/politan-row-chicago-c...
1,ricks-diner-chicago,"[{'alias': 'diners', 'title': 'Diners'}]","{'latitude': 41.9452096002495, 'longitude': -8...",(773) 688-5585,NaN,9Gei7OFl8mtKBzsSQvgQqQ,https://s3-media3.fl.yelpcdn.com/bphoto/b1R6sc...,True,True,"{'address1': '7544 W Addison St', 'address2': ...",NaN,Rick's Diner,+17736885585,[https://s3-media3.fl.yelpcdn.com/bphoto/b1R6s...,NaN,4.5,14,NaN,"[delivery, pickup]",https://www.yelp.com/biz/ricks-diner-chicago?a...
2,el-puerto-mariscos-and-grill-chicago,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 41.9688, 'longitude': -87.69808}",(773) 930-3160,"[{'open': [{'is_overnight': False, 'start': '0...",VS6BQRqCm5jZ-7jXMTwfSg,https://s3-media3.fl.yelpcdn.com/bphoto/uSPA4T...,True,False,"{'address1': '2752 W Lawrence Ave', 'address2'...",NaN,El Puerto Mariscos & Grill,+17739303160,[https://s3-media3.fl.yelpcdn.com/bphoto/uSPA4...,NaN,3.0,5,NaN,"[pickup, delivery]",https://www.yelp.com/biz/el-puerto-mariscos-an...
3,d-fruta-la-vida-chicago,"[{'alias': 'juicebars', 'title': 'Juice Bars &...","{'latitude': 41.924243927002, 'longitude': -87...",(773) 905-7155,"[{'open': [{'is_overnight': False, 'start': '0...",hv9IJpOjOkigqN7n8BeS4A,https://s3-media1.fl.yelpcdn.com/bphoto/LxGL3P...,True,False,"{'address1': '3739 W Fullerton Ave', 'address2...",{'url': 'https://www.yelp.com/raq/d-fruta-la-v...,D Fruta La Vida,+17739057155,[https://s3-media1.fl.yelpcdn.com/bphoto/LxGL3...,$,4.0,50,NaN,"[pickup, delivery]",https://www.yelp.com/biz/d-fruta-la-vida-chica...
4,las-islas-marias-chicago-3,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 41.913829, 'longitude': -87.7453618}",(773) 637-8233,"[{'open': [{'is_overnight': False, 'start': '0...",k6-CqrC7IiIsMpypP4vaZw,https://s3-media3.fl.yelpcdn.com/bphoto/oi5mri...,False,False,"{'address1': '4770 W Grand Ave', 'address2': '...",NaN,Las Islas Marias,+17736378233,[https://s3-media3.fl.yelpcdn.com/bphoto/oi5mr...,$$,3.0,43,NaN,[],https://www.yelp.com/biz/las-islas-marias-chic...


In [17]:
#Split out location field from data frame into seperate columns
responses_location_df = responses_df['location'].apply(pd.Series)
responses_location_df.head()

,address1,address2,address3,city,zip_code,country,state,display_address,cross_streets
0,111 N Aberdeen St,None,,Chicago,60607,US,IL,"[111 N Aberdeen St, Chicago, IL 60607]",Adams St & Monroe St
1,7544 W Addison St,,None,Chicago,60634,US,IL,"[7544 W Addison St, Chicago, IL 60634]",Oleander Ave & Oriole Ave
2,2752 W Lawrence Ave,,None,Chicago,60625,US,IL,"[2752 W Lawrence Ave, Chicago, IL 60625]",Fairfield Ave & California Ave
3,3739 W Fullerton Ave,None,,Chicago,60647,US,IL,"[3739 W Fullerton Ave, Chicago, IL 60647]",Hamlin Ave & Lawndale Ave
4,4770 W Grand Ave,,,Chicago,60639,US,IL,"[4770 W Grand Ave, Chicago, IL 60639]",Keating Ave & Illinois Route 50


In [18]:
#Append seperated location columns to original Yelp dataframe
location_responses_combined_df = pd.concat([responses_df.drop(['location'], axis=1), responses_df['location'].apply(pd.Series)], axis=1)
location_responses_combined_df.head()

,alias,categories,coordinates,display_phone,hours,id,image_url,is_claimed,is_closed,messaging,...,url,address1,address2,address3,city,zip_code,country,state,display_address,cross_streets
0,politan-row-chicago-chicago-2,"[{'alias': 'food_court', 'title': 'Food Court'}]","{'latitude': 41.8832451, 'longitude': -87.654196}",(312) 278-3040,"[{'open': [{'is_overnight': False, 'start': '0...",rsZ6Qd_lxW2NEKK5vy8ZVg,https://s3-media2.fl.yelpcdn.com/bphoto/LYBaBR...,True,False,{'url': 'https://www.yelp.com/raq/politan-row-...,...,https://www.yelp.com/biz/politan-row-chicago-c...,111 N Aberdeen St,None,,Chicago,60607,US,IL,"[111 N Aberdeen St, Chicago, IL 60607]",Adams St & Monroe St
1,ricks-diner-chicago,"[{'alias': 'diners', 'title': 'Diners'}]","{'latitude': 41.9452096002495, 'longitude': -8...",(773) 688-5585,NaN,9Gei7OFl8mtKBzsSQvgQqQ,https://s3-media3.fl.yelpcdn.com/bphoto/b1R6sc...,True,True,NaN,...,https://www.yelp.com/biz/ricks-diner-chicago?a...,7544 W Addison St,,None,Chicago,60634,US,IL,"[7544 W Addison St, Chicago, IL 60634]",Oleander Ave & Oriole Ave
2,el-puerto-mariscos-and-grill-chicago,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 41.9688, 'longitude': -87.69808}",(773) 930-3160,"[{'open': [{'is_overnight': False, 'start': '0...",VS6BQRqCm5jZ-7jXMTwfSg,https://s3-media3.fl.yelpcdn.com/bphoto/uSPA4T...,True,False,NaN,...,https://www.yelp.com/biz/el-puerto-mariscos-an...,2752 W Lawrence Ave,,None,Chicago,60625,US,IL,"[2752 W Lawrence Ave, Chicago, IL 60625]",Fairfield Ave & California Ave
3,d-fruta-la-vida-chicago,"[{'alias': 'juicebars', 'title': 'Juice Bars &...","{'latitude': 41.924243927002, 'longitude': -87...",(773) 905-7155,"[{'open': [{'is_overnight': False, 'start': '0...",hv9IJpOjOkigqN7n8BeS4A,https://s3-media1.fl.yelpcdn.com/bphoto/LxGL3P...,True,False,{'url': 'https://www.yelp.com/raq/d-fruta-la-v...,...,https://www.yelp.com/biz/d-fruta-la-vida-chica...,3739 W Fullerton Ave,None,,Chicago,60647,US,IL,"[3739 W Fullerton Ave, Chicago, IL 60647]",Hamlin Ave & Lawndale Ave
4,las-islas-marias-chicago-3,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 41.913829, 'longitude': -87.7453618}",(773) 637-8233,"[{'open': [{'is_overnight': False, 'start': '0...",k6-CqrC7IiIsMpypP4vaZw,https://s3-media3.fl.yelpcdn.com/bphoto/oi5mri...,False,False,NaN,...,https://www.yelp.com/biz/las-islas-marias-chic...,4770 W Grand Ave,,,Chicago,60639,US,IL,"[4770 W Grand Ave, Chicago, IL 60639]",Keating Ave & Illinois Route 50


In [19]:
#Reorder columns in Yelp dataframe
cleaned_responses_df = location_responses_combined_df.filter(['alias', 'categories', 'coordinates', 'display_phone', 'hours', 'id',
       'image_url', 'is_claimed', 'is_closed','display_address', 'address1', 'address2', 'address3', 'city','state','zip_code','country','name','phone','price','rating','review_count','transactions','url'], axis=1)

#Rename columns to align to database fields
cleaned_responses_df = cleaned_responses_df.rename(columns={"id": "yelp_id",
                                                         "name": "business_name", "url":"yelp_url"})

cleaned_responses_df.head()

,alias,categories,coordinates,display_phone,hours,yelp_id,image_url,is_claimed,is_closed,display_address,...,state,zip_code,country,business_name,phone,price,rating,review_count,transactions,yelp_url
0,politan-row-chicago-chicago-2,"[{'alias': 'food_court', 'title': 'Food Court'}]","{'latitude': 41.8832451, 'longitude': -87.654196}",(312) 278-3040,"[{'open': [{'is_overnight': False, 'start': '0...",rsZ6Qd_lxW2NEKK5vy8ZVg,https://s3-media2.fl.yelpcdn.com/bphoto/LYBaBR...,True,False,"[111 N Aberdeen St, Chicago, IL 60607]",...,IL,60607,US,Politan Row Chicago,+13122783040,NaN,4.5,9,[],https://www.yelp.com/biz/politan-row-chicago-c...
1,ricks-diner-chicago,"[{'alias': 'diners', 'title': 'Diners'}]","{'latitude': 41.9452096002495, 'longitude': -8...",(773) 688-5585,NaN,9Gei7OFl8mtKBzsSQvgQqQ,https://s3-media3.fl.yelpcdn.com/bphoto/b1R6sc...,True,True,"[7544 W Addison St, Chicago, IL 60634]",...,IL,60634,US,Rick's Diner,+17736885585,NaN,4.5,14,"[delivery, pickup]",https://www.yelp.com/biz/ricks-diner-chicago?a...
2,el-puerto-mariscos-and-grill-chicago,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 41.9688, 'longitude': -87.69808}",(773) 930-3160,"[{'open': [{'is_overnight': False, 'start': '0...",VS6BQRqCm5jZ-7jXMTwfSg,https://s3-media3.fl.yelpcdn.com/bphoto/uSPA4T...,True,False,"[2752 W Lawrence Ave, Chicago, IL 60625]",...,IL,60625,US,El Puerto Mariscos & Grill,+17739303160,NaN,3.0,5,"[pickup, delivery]",https://www.yelp.com/biz/el-puerto-mariscos-an...
3,d-fruta-la-vida-chicago,"[{'alias': 'juicebars', 'title': 'Juice Bars &...","{'latitude': 41.924243927002, 'longitude': -87...",(773) 905-7155,"[{'open': [{'is_overnight': False, 'start': '0...",hv9IJpOjOkigqN7n8BeS4A,https://s3-media1.fl.yelpcdn.com/bphoto/LxGL3P...,True,False,"[3739 W Fullerton Ave, Chicago, IL 60647]",...,IL,60647,US,D Fruta La Vida,+17739057155,$,4.0,50,"[pickup, delivery]",https://www.yelp.com/biz/d-fruta-la-vida-chica...
4,las-islas-marias-chicago-3,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 41.913829, 'longitude': -87.7453618}",(773) 637-8233,"[{'open': [{'is_overnight': False, 'start': '0...",k6-CqrC7IiIsMpypP4vaZw,https://s3-media3.fl.yelpcdn.com/bphoto/oi5mri...,False,False,"[4770 W Grand Ave, Chicago, IL 60639]",...,IL,60639,US,Las Islas Marias,+17736378233,$$,3.0,43,[],https://www.yelp.com/biz/las-islas-marias-chic...


In [20]:
#Export to CSV for import at later time
if not os.path.isfile('yelp_data.csv'):
   cleaned_responses_df.to_csv('yelp_data.csv', header='column_names', index=False)
else: # else it exists so append without writing the header
   cleaned_responses_df.to_csv('yelp_data.csv', mode='a', header=False, index=False)

## Importing API Results to MySQL

In [21]:
#Read in yelp data CSV, give index column the name ID, increment index field by 1
restaurant_inspection_df = pd.read_csv("restaurant_inspection_data.csv", header=0)
restaurant_inspection_df.index.name = 'id'
restaurant_inspection_df.index += 1 
restaurant_inspection_df

,inspection_id,dba_name,aka_name,license_no,facility_type,risk,address,city,state,zip,...,results,violations,latitude,longitude,location,historical_wards,zip_codes,community_areas,census_tracts,wards
id,,,,,,,,,,,,,,,,,,,,,
1,2288615,SABROSO,SABROSO,2664160.0,Restaurant,Risk 1 (High),624 N ASHLAND AVE,CHICAGO,IL,60622.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.893250,-87.667356,"{'latitude': '41.893249885934495', 'longitude'...",24.0,21560.0,25.0,520.0,41.0
2,2288595,POLITAN ROW,POLITAN ROW,2621900.0,Restaurant,Risk 3 (Low),111 N ABERDEEN ST,CHICAGO,IL,60607.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.883370,-87.654261,"{'latitude': '41.883370342897926', 'longitude'...",41.0,14917.0,29.0,63.0,46.0
3,2288613,ANTHONY'S DINER,ANTHONY'S DINER,2664063.0,Restaurant,Risk 1 (High),7544 W ADDISON ST,CHICAGO,IL,60634.0,...,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.945226,-87.816074,"{'latitude': '41.945225909077436', 'longitude'...",39.0,22254.0,17.0,392.0,19.0
4,2288630,EL NUEVO PUERTO MARISCOS AND GRILL,EL NUEVO PUERTO MARISCOS AND GRILL,2664156.0,Restaurant,Risk 1 (High),2752 W LAWRENCE AVE,CHICAGO,IL,60625.0,...,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.968645,-87.698286,"{'latitude': '41.968644613011605', 'longitude'...",46.0,21849.0,6.0,28.0,24.0
5,2288623,D FRUTA LA VIDA LLC,D FRUTA LA VIDA,2398055.0,Restaurant,Risk 1 (High),3739 W FULLERTON AVE,CHICAGO,IL,60647.0,...,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.924443,-87.721033,"{'latitude': '41.924442829878934', 'longitude'...",15.0,22535.0,23.0,74.0,12.0
6,2288589,LAS ISLAS MARIAS,LAS ISLAS MARIAS,1095992.0,Restaurant,Risk 1 (High),4770 W GRAND AVE,CHICAGO,IL,60639.0,...,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.913999,-87.745864,"{'latitude': '41.91399920930635', 'longitude':...",7.0,22615.0,26.0,62.0,44.0
7,2288628,TOM AND JERRY'S,TOM AND JERRY'S,2664659.0,Restaurant,Risk 1 (High),6359 W MONTROSE AVE,CHICAGO,IL,60634.0,...,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.960108,-87.786742,"{'latitude': '41.960108059177585', 'longitude'...",25.0,22254.0,15.0,131.0,19.0
8,2288643,RANGOLI,RANGOLI,2506262.0,Restaurant,Risk 1 (High),2421 W NORTH AVE,CHICAGO,IL,60647.0,...,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.910200,-87.688125,"{'latitude': '41.91019989714011', 'longitude':...",24.0,22535.0,25.0,302.0,41.0
9,2288612,DEVIL DAWGS,DEVIL DAWGS,2622063.0,Restaurant,Risk 1 (High),939 W BELMONT AVE,CHICAGO,IL,60657.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.939813,-87.653091,"{'latitude': '41.939813147048625', 'longitude'...",38.0,4449.0,57.0,681.0,25.0


In [22]:
#Read in yelp data CSV, give index column the name ID, increment index field by 1
yelp_business_info_df = pd.read_csv("yelp_data.csv", header=0)
yelp_business_info_df.index.name = 'id'
yelp_business_info_df.index += 1 
yelp_business_info_df

,alias,categories,coordinates,display_phone,hours,yelp_id,image_url,is_claimed,is_closed,display_address,...,state,zip_code,country,business_name,phone,price,rating,review_count,transactions,yelp_url
id,,,,,,,,,,,,,,,,,,,,,
1,politan-row-chicago-chicago-2,"[{'alias': 'food_court', 'title': 'Food Court'}]","{'latitude': 41.8832451, 'longitude': -87.654196}",(312) 278-3040,"[{'open': [{'is_overnight': False, 'start': '0...",rsZ6Qd_lxW2NEKK5vy8ZVg,https://s3-media2.fl.yelpcdn.com/bphoto/LYBaBR...,True,False,"['111 N Aberdeen St', 'Chicago, IL 60607']",...,IL,60607,US,Politan Row Chicago,1.312278e+10,NaN,4.5,9,[],https://www.yelp.com/biz/politan-row-chicago-c...
2,ricks-diner-chicago,"[{'alias': 'diners', 'title': 'Diners'}]","{'latitude': 41.9452096002495, 'longitude': -8...",(773) 688-5585,NaN,9Gei7OFl8mtKBzsSQvgQqQ,https://s3-media3.fl.yelpcdn.com/bphoto/b1R6sc...,True,True,"['7544 W Addison St', 'Chicago, IL 60634']",...,IL,60634,US,Rick's Diner,1.773689e+10,NaN,4.5,14,"['delivery', 'pickup']",https://www.yelp.com/biz/ricks-diner-chicago?a...
3,el-puerto-mariscos-and-grill-chicago,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 41.9688, 'longitude': -87.69808}",(773) 930-3160,"[{'open': [{'is_overnight': False, 'start': '0...",VS6BQRqCm5jZ-7jXMTwfSg,https://s3-media3.fl.yelpcdn.com/bphoto/uSPA4T...,True,False,"['2752 W Lawrence Ave', 'Chicago, IL 60625']",...,IL,60625,US,El Puerto Mariscos & Grill,1.773930e+10,NaN,3.0,5,"['pickup', 'delivery']",https://www.yelp.com/biz/el-puerto-mariscos-an...
4,d-fruta-la-vida-chicago,"[{'alias': 'juicebars', 'title': 'Juice Bars &...","{'latitude': 41.924243927002, 'longitude': -87...",(773) 905-7155,"[{'open': [{'is_overnight': False, 'start': '0...",hv9IJpOjOkigqN7n8BeS4A,https://s3-media1.fl.yelpcdn.com/bphoto/LxGL3P...,True,False,"['3739 W Fullerton Ave', 'Chicago, IL 60647']",...,IL,60647,US,D Fruta La Vida,1.773906e+10,$,4.0,50,"['pickup', 'delivery']",https://www.yelp.com/biz/d-fruta-la-vida-chica...
5,las-islas-marias-chicago-3,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 41.913829, 'longitude': -87.7453618}",(773) 637-8233,"[{'open': [{'is_overnight': False, 'start': '0...",k6-CqrC7IiIsMpypP4vaZw,https://s3-media3.fl.yelpcdn.com/bphoto/oi5mri...,False,False,"['4770 W Grand Ave', 'Chicago, IL 60639']",...,IL,60639,US,Las Islas Marias,1.773638e+10,$$,3.0,43,[],https://www.yelp.com/biz/las-islas-marias-chic...
6,tom-and-jerrys-chicago-3,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 41.9598324597632, 'longitude': -8...",(773) 282-1111,"[{'open': [{'is_overnight': False, 'start': '1...",hmX4VH9WgDrEL9HJKcCL5A,https://s3-media1.fl.yelpcdn.com/bphoto/h_iF3S...,True,False,"['6359 W Montrose Ave', 'Chicago, IL 60634']",...,IL,60634,US,Tom & Jerry's,1.773282e+10,$,3.0,77,"['delivery', 'pickup']",https://www.yelp.com/biz/tom-and-jerrys-chicag...
7,rangoli-chicago,"[{'alias': 'indpak', 'title': 'Indian'}]","{'latitude': 41.910198, 'longitude': -87.68809}",(773) 697-7114,"[{'open': [{'is_overnight': False, 'start': '1...",1GiybV2faVpTt1zw13CzLA,https://s3-media1.fl.yelpcdn.com/bphoto/meetfq...,True,False,"['2421 W North Ave', 'Chicago, IL 60647']",...,IL,60647,US,Rangoli,1.773698e+10,$$,4.0,606,"['pickup', 'delivery']",https://www.yelp.com/biz/rangoli-chicago?adjus...
8,birrieria-la-tapatia-de-ocotlan-chicago,"[{'alias': 'mexican', 'title': 'Mexican'}]","{'latitude': 41.8516006469727, 'longitude': -8...",(773) 247-7914,"[{'open': [{'is_overnight': False, 'start': '0...",XW6EiWtKY4-ZBq7TYUowiQ,https://s3-media3.fl.yelpcdn.com/bphoto/V5yOtC...,False,False,"['2861 W Cermak Rd', 'Chicago, IL 60623']",...,IL,60623,US,Birrieria La Tapatia De Ocotlan,1.773248e+10,NaN,5.0,3,[],https://www.yelp.com/biz/birrieria-la-tapatia-...
9,sugar-factory-american-brasserie-chicago,"[{'alias': 'candy', 'title': 'Candy Stores'}, ...","{'latitude': 41.8913867352473, 'longitude': -8...",(312) 946-2002,"[{'open': [{'is_overnight': False, 'start': '1...",MRZRq78_b

In [23]:
#Read in license/yelp ID data CSV, give index column the name ID, increment index field by 1
business_info_df = pd.read_csv("id_data.csv", header=0)
business_info_df.index.name = 'id'
business_info_df.index += 1 
business_info_df

,license_no,yelp_id,inspection_name,inspection_address,yelp_name,yelp_address
id,,,,,,
1,2664160.0,0,SABROSO,624 N ASHLAND AVE,NaN,NaN
2,2621900.0,rsZ6Qd_lxW2NEKK5vy8ZVg,POLITAN ROW,111 N ABERDEEN ST,Politan Row Chicago,111 N Aberdeen St
3,2664063.0,9Gei7OFl8mtKBzsSQvgQqQ,ANTHONY'S DINER,7544 W ADDISON ST,Rick's Diner,7544 W Addison St
4,2664156.0,VS6BQRqCm5jZ-7jXMTwfSg,EL NUEVO PUERTO MARISCOS AND GRILL,2752 W LAWRENCE AVE,El Puerto Mariscos & Grill,2752 W Lawrence Ave
5,2398055.0,hv9IJpOjOkigqN7n8BeS4A,D FRUTA LA VIDA LLC,3739 W FULLERTON AVE,D Fruta La Vida,3739 W Fullerton Ave
6,1095992.0,k6-CqrC7IiIsMpypP4vaZw,LAS ISLAS MARIAS,4770 W GRAND AVE,Las Islas Marias,4770 W Grand Ave
7,2664659.0,hmX4VH9WgDrEL9HJKcCL5A,TOM AND JERRY'S,6359 W MONTROSE AVE,Tom & Jerry's,6359 W Montrose Ave
8,2506262.0,1GiybV2faVpTt1zw13CzLA,RANGOLI,2421 W NORTH AVE,Rangoli,2421 W North Ave
9,2622063.0,0,DEVIL DAWGS,939 W BELMONT AVE,NaN,NaN


In [24]:
#Create connection to the MySQL database
connection_string = "root:Schwornje1@localhost/chicago_yelp"
engine = create_engine(f'mysql+pymysql://{connection_string}')

In [25]:
#Check table names
engine.table_names()

['business_info', 'food_inspection', 'yelp_business_info']

In [26]:
#Insert yelp data dataframe into the appropriate MySQL table
yelp_business_info_df.to_sql(name='yelp_business_info', con=engine, if_exists='append', index=True)

In [27]:
#Insert business id dataframe into the appropriate MySQL table
business_info_df.to_sql(name='business_info', con=engine, if_exists='append', index=True)

In [28]:
#Insert inspection data dataframe into the appropriate MySQL table
restaurant_inspection_df.to_sql(name='food_inspection', con=engine, if_exists='append', index=True)